# GPU Cluster Spot Resource Dataset Analysis (Detailed Version)

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

OUTDIR = Path("outputs")
FIGDIR = OUTDIR / "figures"
TBLDIR = OUTDIR / "tables"
OUTDIR.mkdir(exist_ok=True, parents=True)
FIGDIR.mkdir(exist_ok=True, parents=True)
TBLDIR.mkdir(exist_ok=True, parents=True)

# Load the datasets
print("Loading datasets...")
# node_df = pd.read_csv("node_info_df.csv")
# job_df = pd.read_csv("job_info_df.csv")

node_df = pd.read_csv('/kaggle/input/alibaba-gpu-cluster-spot-resource-dataset/node_info_df.csv')
job_df = pd.read_csv('/kaggle/input/alibaba-gpu-cluster-spot-resource-dataset/job_info_df.csv')

def human(n):
    for unit in ["", "K", "M", "B", "T"]:
        if abs(n) < 1000.0:
            return f"{n:,.2f}{unit}"
        n /= 1000.0
    return f"{n:,.2f}P"

Loading datasets...


In [2]:
# Basic structure
print("Nodes rows and columns:", node_df.shape)
print("Jobs rows and columns:", job_df.shape)

# Data quality checks
node_nulls = node_df.isna().sum().sort_values(ascending=False)
job_nulls = job_df.isna().sum().sort_values(ascending=False)
node_nulls.to_csv(TBLDIR / "node_null_counts.csv", header=["nulls"])
job_nulls.to_csv(TBLDIR / "job_null_counts.csv", header=["nulls"])

dupe_jobs = job_df.duplicated(subset=["job_name"]).sum() if "job_name" in job_df.columns else np.nan
print("Duplicate job_name count:", dupe_jobs)

# Enrich jobs
print("Enriching jobs...")
if "gpu_request" not in job_df.columns or "worker_num" not in job_df.columns:
    raise ValueError("gpu_request and worker_num must exist in job_info_df.csv")

if "cpu_request" not in job_df.columns:
    raise ValueError("cpu_request must exist in job_info_df.csv")

if "duration" not in job_df.columns:
    raise ValueError("duration must exist in job_info_df.csv")

if "submit_time" not in job_df.columns:
    raise ValueError("submit_time must exist in job_info_df.csv")

job_df = job_df.copy()
job_df["gpu_demand"] = job_df["gpu_request"] * job_df["worker_num"]
job_df["gpu_seconds"] = job_df["gpu_demand"] * job_df["duration"]
job_df["cpu_seconds"] = job_df["cpu_request"] * job_df["duration"]
job_df["submit_day"] = (job_df["submit_time"] // 86400).astype(int)
job_df["submit_hour"] = (job_df["submit_time"] // 3600).astype(int)
job_df["is_multi_worker"] = job_df["worker_num"] > 1
job_df["duration_hours"] = job_df["duration"] / 3600.0

Nodes rows and columns: (4278, 4)
Jobs rows and columns: (466867, 9)
Duplicate job_name count: 0
Enriching jobs...


In [3]:
# Fleet summary
print("Summarizing fleet...")
if not {"gpu_capacity_num","gpu_model"}.issubset(node_df.columns):
    raise ValueError("node_info_df.csv must include gpu_capacity_num and gpu_model")

fleet_cards = int(node_df["gpu_capacity_num"].sum())
fleet_nodes = len(node_df)
fleet_models = node_df["gpu_model"].nunique()

by_model = (
    node_df.groupby("gpu_model")
    .agg(nodes=("node_name","count"),
         total_gpus=("gpu_capacity_num","sum"),
         median_cpu=("cpu_num","median"),
         max_gpus=("gpu_capacity_num","max"))
    .sort_values("total_gpus", ascending=False)
)
by_model.to_csv(TBLDIR / "fleet_by_model.csv")

print(f"Total nodes {fleet_nodes}, total GPU cards {fleet_cards}, GPU models {fleet_models}")

Summarizing fleet...
Total nodes 4278, total GPU cards 10412, GPU models 6


In [4]:
# Workload summary
print("Summarizing workload...")
total_jobs = len(job_df)
unique_orgs = job_df["organization"].nunique() if "organization" in job_df.columns else np.nan
coverage_days = float(job_df["submit_time"].max()) / 86400.0

by_type = (
    job_df.groupby("job_type", dropna=False)
    .agg(jobs=("job_name","count"),
         gpu_seconds=("gpu_seconds","sum"),
         cpu_seconds=("cpu_seconds","sum"),
         median_duration=("duration","median"),
         p90_duration=("duration", lambda s: float(np.quantile(s, 0.90))))
    .sort_values("jobs", ascending=False)
)
by_type.to_csv(TBLDIR / "job_type_summary.csv")

medians = job_df[["cpu_request","gpu_request","worker_num","duration"]].median().rename("median")
medians.to_csv(TBLDIR / "job_medians.csv", header=True)

multi_worker_share = float((job_df["worker_num"] > 1).mean())
max_workers = int(job_df["worker_num"].max())

Summarizing workload...


In [5]:
# Totals
total_gpu_seconds = float(job_df["gpu_seconds"].sum())
total_cpu_seconds = float(job_df["cpu_seconds"].sum())
gpu_years = total_gpu_seconds / (3600 * 24 * 365)
cpu_years = total_cpu_seconds / (3600 * 24 * 365)

# Demand by GPU model in jobs if available
job_model_col = None
for c in ["gpu_model","requested_gpu_model","gpu_type"]:
    if c in job_df.columns:
        job_model_col = c
        break

if job_model_col is not None:
    demand_model = (
        job_df.groupby(job_model_col)
        .agg(jobs=("job_name","count"),
             gpu_demand=("gpu_demand","sum"),
             gpu_seconds=("gpu_seconds","sum"),
             median_gpu_req=("gpu_request","median"))
        .sort_values("jobs", ascending=False)
    )
    demand_model["jobs_share"] = demand_model["jobs"] / demand_model["jobs"].sum()
    demand_model["gpu_seconds_share"] = demand_model["gpu_seconds"] / demand_model["gpu_seconds"].sum()
    demand_model.to_csv(TBLDIR / "demand_by_model.csv")
else:
    print("Job file has no GPU model column, skipping demand by model table.")
    demand_model = None

In [6]:
# Organization table
if "organization" in job_df.columns:
    org_tbl = (
        job_df.groupby("organization")
        .agg(jobs=("job_name","count"),
             gpu_seconds=("gpu_seconds","sum"),
             gpu_demand=("gpu_demand","sum"),
             median_duration=("duration","median"))
        .sort_values("gpu_seconds", ascending=False)
    )
    org_tbl.to_csv(TBLDIR / "org_by_gpu_seconds.csv")
else:
    org_tbl = None

# Daily and hourly tables
daily_tbl = (
    job_df.groupby("submit_day")
    .agg(jobs=("job_name","count"),
         hp_jobs=("job_type", lambda s: int((s == "HP").sum()) if s.notna().any() else 0),
         spot_jobs=("job_type", lambda s: int((s == "Spot").sum()) if s.notna().any() else 0),
         gpu_seconds=("gpu_seconds","sum"),
         gpu_demand=("gpu_demand","sum"))
    .reset_index()
)
daily_tbl.to_csv(TBLDIR / "daily_summary.csv", index=False)

hourly_tbl = (
    job_df.groupby("submit_hour")
    .agg(jobs=("job_name","count"),
         gpu_seconds=("gpu_seconds","sum"),
         gpu_demand=("gpu_demand","sum"))
    .reset_index()
)
hourly_tbl.to_csv(TBLDIR / "hourly_summary.csv", index=False)

In [7]:
# Quantiles and long tail
quantiles = job_df["duration"].quantile([0.5, 0.9, 0.95, 0.99]).rename("duration_seconds")
quantiles.to_csv(TBLDIR / "duration_quantiles.csv", header=True)

top_long = job_df.sort_values("duration", ascending=False).head(200)
cols_export = [c for c in ["job_name","organization","job_type","duration","gpu_request","cpu_request","worker_num","gpu_seconds","submit_time"] if c in job_df.columns]
top_long[cols_export].to_csv(TBLDIR / "longest_jobs_top200.csv", index=False)

top_gpu = job_df.sort_values("gpu_seconds", ascending=False).head(200)
top_gpu[cols_export].to_csv(TBLDIR / "heaviest_gpu_jobs_top200.csv", index=False)

# Correlations
corr_cols = [c for c in ["cpu_request","gpu_request","worker_num","duration","gpu_demand"] if c in job_df.columns]
corr = job_df[corr_cols].corr(numeric_only=True)
corr.to_csv(TBLDIR / "correlations.csv")

# Worker gang distribution
worker_dist = job_df["worker_num"].value_counts().sort_index()
worker_dist.to_csv(TBLDIR / "worker_count_distribution.csv", header=["jobs"])

In [8]:
# Figures
print("Creating figures...")

# 1. Fleet GPUs by model
plt.figure()
node_df.groupby("gpu_model")["gpu_capacity_num"].sum().sort_values(ascending=False).plot(kind="bar", title="Total GPUs by model")
plt.ylabel("GPU cards")
plt.tight_layout()
plt.savefig(FIGDIR / "fleet_gpus_by_model.png")
plt.close()

# 2. Job duration histogram, capped at seven days for readability
plt.figure()
job_df["duration"].clip(upper=3600*24*7).plot(kind="hist", bins=60, title="Job duration capped at seven days")
plt.xlabel("Seconds")
plt.tight_layout()
plt.savefig(FIGDIR / "job_duration_hist.png")
plt.close()

# 3. Daily GPU seconds line
plt.figure()
daily_tbl.plot(x="submit_day", y="gpu_seconds", kind="line", title="Daily GPU seconds")
plt.xlabel("Day index")
plt.ylabel("GPU seconds")
plt.tight_layout()
plt.savefig(FIGDIR / "daily_gpu_seconds.png")
plt.close()

# 4. CPU versus GPU request scatter, sample for speed
plt.figure()
sample_n = min(100000, len(job_df))
job_df.sample(sample_n, random_state=42).plot(x="gpu_request", y="cpu_request", kind="scatter", title="CPU request versus GPU request")
plt.tight_layout()
plt.savefig(FIGDIR / "cpu_vs_gpu_request.png")
plt.close()

# 5. Worker count histogram
plt.figure()
job_df["worker_num"].plot(kind="hist", bins=60, title="Worker count distribution")
plt.xlabel("Worker count")
plt.tight_layout()
plt.savefig(FIGDIR / "worker_count_hist.png")
plt.close()

# 6. Organization top twenty by GPU seconds bar
if org_tbl is not None:
    plt.figure()
    org_tbl.head(20)["gpu_seconds"].plot(kind="bar", title="Top orgs by GPU seconds")
    plt.ylabel("GPU seconds")
    plt.tight_layout()
    plt.savefig(FIGDIR / "org_top20_gpu_seconds.png")
    plt.close()

# 7. GPU seconds by model from jobs if available
if demand_model is not None:
    plt.figure()
    demand_model["gpu_seconds"].sort_values(ascending=False).plot(kind="bar", title="GPU seconds by model in jobs")
    plt.ylabel("GPU seconds")
    plt.tight_layout()
    plt.savefig(FIGDIR / "gpu_seconds_by_model_jobs.png")
    plt.close()

Creating figures...


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [9]:
# Summary text file
with open(OUTDIR / "summary.txt", "w", encoding="utf-8") as f:
    f.write("Dataset summary\n")
    f.write(f"Nodes {fleet_nodes}, GPU cards {fleet_cards}, GPU models {fleet_models}\n")
    f.write(f"Jobs {total_jobs}, Organizations {unique_orgs}\n")
    f.write(f"Trace span days {coverage_days:.2f}\n")
    f.write(f"Total GPU seconds {human(total_gpu_seconds)}, GPU years {gpu_years:.2f}\n")
    f.write(f"Total CPU seconds {human(total_cpu_seconds)}, CPU years {cpu_years:.2f}\n")
    f.write(f"Multi worker share {multi_worker_share:.2%}, Max workers {max_workers}\n")

print("Done. Tables in", TBLDIR, "Figures in", FIGDIR, "Summary in", OUTDIR / "summary.txt")


Done. Tables in outputs\tables Figures in outputs\figures Summary in outputs\summary.txt


# Advanced

In [ ]:
# Extra dataset analysis that adds concurrency, capacity ratios, diurnal patterns, Pareto and Gini, buckets, heatmaps, and outlier finds.

import math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

OUTDIR = Path("outputs_extra")
FIGDIR = OUTDIR / "figures"
TBLDIR = OUTDIR / "tables"
OUTDIR.mkdir(exist_ok=True, parents=True)
FIGDIR.mkdir(exist_ok=True, parents=True)
TBLDIR.mkdir(exist_ok=True, parents=True)

print("Loading datasets...")
# node_df = pd.read_csv("node_info_df.csv")
# job_df = pd.read_csv("job_info_df.csv")

node_df = pd.read_csv('/kaggle/input/alibaba-gpu-cluster-spot-resource-dataset/node_info_df.csv')
job_df = pd.read_csv('/kaggle/input/alibaba-gpu-cluster-spot-resource-dataset/job_info_df.csv')

print("Enriching jobs...")
job_df = job_df.copy()
job_df["end_time"] = job_df["submit_time"] + job_df["duration"]
job_df["gpu_demand"] = job_df["gpu_request"] * job_df["worker_num"]
job_df["gpu_seconds"] = job_df["gpu_demand"] * job_df["duration"]
job_df["cpu_seconds"] = job_df["cpu_request"] * job_df["duration"]
job_df["hour_of_day"] = ((job_df["submit_time"] % 86400) // 3600).astype(int)
job_df["submit_day"] = (job_df["submit_time"] // 86400).astype(int)
job_df["is_multi_worker"] = job_df["worker_num"] > 1
job_df["cpu_per_gpu"] = job_df["cpu_request"] / job_df["gpu_request"].replace(0, np.nan)
job_df["duration_hours"] = job_df["duration"] / 3600.0

Loading datasets...
Enriching jobs...


In [11]:
# Capacity constants
total_cards = int(node_df["gpu_capacity_num"].sum())
cap_gpu_seconds_per_day = total_cards * 86400

# 1. Concurrency time series at hourly resolution for jobs and GPU demand
print("Computing hourly concurrency...")
max_hour = int(np.ceil(job_df["end_time"].max() / 3600.0))
start_hour = (job_df["submit_time"] // 3600).astype(int).values
end_hour = ((job_df["end_time"] - 1) // 3600).astype(int).values
gpu_demand = job_df["gpu_demand"].values
ones = np.ones(len(job_df), dtype=float)

def diff_to_series(starts, ends, weights, length):
    diff = np.zeros(length + 2, dtype=float)
    np.add.at(diff, starts, weights)
    np.add.at(diff, ends + 1, -weights)
    series = np.cumsum(diff)[: length + 1]
    return series

gpu_conc = diff_to_series(start_hour, end_hour, gpu_demand, max_hour)
job_conc = diff_to_series(start_hour, end_hour, ones, max_hour)

hour_idx = np.arange(len(gpu_conc))
hourly = pd.DataFrame({
    "hour": hour_idx,
    "jobs_active": job_conc,
    "gpus_active_requested": gpu_conc,
    "capacity_gpus": total_cards,
    "demand_capacity_ratio": gpu_conc / np.maximum(total_cards, 1)
})
hourly.to_csv(TBLDIR / "hourly_concurrency.csv", index=False)

plt.figure()
hourly.plot(x="hour", y="gpus_active_requested", kind="line", title="Requested GPUs active per hour")
plt.xlabel("Hour since trace start")
plt.ylabel("GPUs requested active")
plt.tight_layout()
plt.savefig(FIGDIR / "hourly_gpu_concurrency.png")
plt.close()

plt.figure()
hourly.plot(x="hour", y="demand_capacity_ratio", kind="line", title="Demand to capacity ratio per hour")
plt.xlabel("Hour since trace start")
plt.ylabel("Ratio")
plt.tight_layout()
plt.savefig(FIGDIR / "hourly_demand_capacity_ratio.png")
plt.close()

Computing hourly concurrency...


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [12]:
# 2. Per model concurrency for top models by GPU seconds
job_model_col = None
for c in ["gpu_model","requested_gpu_model","gpu_type"]:
    if c in job_df.columns:
        job_model_col = c
        break

if job_model_col is not None:
    print("Computing per model concurrency for top models...")
    top_models = job_df.groupby(job_model_col)["gpu_seconds"].sum().sort_values(ascending=False).head(6).index.tolist()
    model_conc = pd.DataFrame({"hour": hour_idx})
    for m in top_models:
        mask = job_df[job_model_col] == m
        m_gpu = diff_to_series(start_hour[mask.values], end_hour[mask.values], gpu_demand[mask.values], max_hour)
        model_conc[str(m)] = m_gpu
        plt.figure()
        pd.DataFrame({"hour": hour_idx, "gpus": m_gpu}).plot(x="hour", y="gpus", kind="line", title=f"Requested GPUs active per hour for {m}")
        plt.xlabel("Hour since trace start")
        plt.ylabel("GPUs requested active")
        plt.tight_layout()
        plt.savefig(FIGDIR / f"hourly_gpu_concurrency_{str(m)}.png")
        plt.close()
    model_conc.to_csv(TBLDIR / "hourly_concurrency_top_models.csv", index=False)
else:
    print("No GPU model column found in jobs for per model concurrency")

Computing per model concurrency for top models...


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [13]:
# 3. Capacity pressure by day, global and per model if available
print("Computing capacity pressure by day...")
daily = (
    job_df.groupby("submit_day")
    .agg(gpu_seconds=("gpu_seconds","sum"),
         jobs=("gpu_seconds","count"))
    .reset_index()
)
daily["capacity_gpu_seconds"] = cap_gpu_seconds_per_day
daily["demand_capacity_ratio"] = daily["gpu_seconds"] / np.maximum(daily["capacity_gpu_seconds"], 1)
daily.to_csv(TBLDIR / "daily_capacity_pressure.csv", index=False)

plt.figure()
daily.plot(x="submit_day", y="demand_capacity_ratio", kind="line", title="Daily demand to capacity ratio")
plt.xlabel("Day since trace start")
plt.ylabel("Ratio")
plt.tight_layout()
plt.savefig(FIGDIR / "daily_demand_capacity_ratio.png")
plt.close()

if job_model_col is not None:
    cap_by_model = node_df.groupby("gpu_model")["gpu_capacity_num"].sum().rename("capacity_cards")
    demand_by_model_day = job_df.groupby(["submit_day", job_model_col])["gpu_seconds"].sum().reset_index()
    top_models = job_df.groupby(job_model_col)["gpu_seconds"].sum().sort_values(ascending=False).head(6).index.tolist()
    demand_top = demand_by_model_day[demand_by_model_day[job_model_col].isin(top_models)]
    demand_piv = demand_top.pivot(index="submit_day", columns=job_model_col, values="gpu_seconds").fillna(0.0)
    ratios = []
    for m in demand_piv.columns:
        cap_cards = float(cap_by_model.get(m, 0.0))
        cap_sec = cap_cards * 86400.0
        ratios.append(demand_piv[m] / max(cap_sec, 1.0))
    ratio_df = pd.concat(ratios, axis=1)
    ratio_df.columns = demand_piv.columns
    ratio_df.to_csv(TBLDIR / "daily_capacity_ratio_top_models.csv")
    plt.figure()
    ratio_df.plot(kind="line", title="Daily demand to capacity ratio by top models")
    plt.xlabel("Day since trace start")
    plt.ylabel("Ratio")
    plt.tight_layout()
    plt.savefig(FIGDIR / "daily_demand_capacity_ratio_by_model.png")
    plt.close()

Computing capacity pressure by day...


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [14]:
# 4. Diurnal patterns by hour of day
print("Computing diurnal patterns...")
hod = (
    job_df.groupby("hour_of_day")
    .agg(jobs=("gpu_seconds","count"),
         gpu_seconds=("gpu_seconds","sum"),
         median_duration=("duration","median"))
    .reset_index()
)
hod.to_csv(TBLDIR / "hour_of_day_patterns.csv", index=False)

plt.figure()
hod.plot(x="hour_of_day", y="jobs", kind="bar", title="Jobs per hour of day")
plt.xlabel("Hour of day")
plt.ylabel("Jobs")
plt.tight_layout()
plt.savefig(FIGDIR / "jobs_per_hour_of_day.png")
plt.close()

plt.figure()
hod.plot(x="hour_of_day", y="gpu_seconds", kind="bar", title="GPU seconds per hour of day")
plt.xlabel("Hour of day")
plt.ylabel("GPU seconds")
plt.tight_layout()
plt.savefig(FIGDIR / "gpu_seconds_per_hour_of_day.png")
plt.close()

Computing diurnal patterns...


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [15]:
# 5. Pareto and Gini across organizations
if "organization" in job_df.columns:
    print("Computing Pareto and Gini across organizations...")
    org_demand = job_df.groupby("organization")["gpu_seconds"].sum().sort_values(ascending=False)
    org_share = org_demand / max(org_demand.sum(), 1.0)
    cum_share = org_share.cumsum()
    pareto = pd.DataFrame({"organization": org_share.index, "gpu_seconds": org_demand.values, "share": org_share.values, "cum_share": cum_share.values})
    pareto.to_csv(TBLDIR / "org_pareto_gpu_seconds.csv", index=False)

    def gini(x):
        x = np.sort(np.asarray(x, dtype=float))
        if x.size == 0:
            return np.nan
        cum = np.cumsum(x)
        rel = cum / cum[-1] if cum[-1] > 0 else cum
        n = x.size
        return 1.0 - 2.0 * np.trapz(rel, dx=1.0) / n

    gini_val = gini(org_demand.values)
    with open(OUTDIR / "summary_extra.txt", "a", encoding="utf-8") as f:
        f.write(f"Gini across organizations by GPU seconds {gini_val:.4f}\n")

    plt.figure()
    x = np.linspace(0, 1, len(org_share), endpoint=True)
    y = cum_share.values
    plt.plot(x, y)
    plt.plot([0,1], [0,1])
    plt.title("Lorenz curve for GPU seconds across organizations")
    plt.xlabel("Cumulative organizations")
    plt.ylabel("Cumulative GPU seconds share")
    plt.tight_layout()
    plt.savefig(FIGDIR / "lorenz_curve_orgs.png")
    plt.close()

Computing Pareto and Gini across organizations...


C:\Users\sulta\AppData\Local\Temp\ipykernel_33392\554595160.py:17: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return 1.0 - 2.0 * np.trapz(rel, dx=1.0) / n


In [16]:
# 6. Buckets for size classes
print("Computing bucketed summaries...")
dur_bins = [0, 60, 600, 3600, 21600, 86400, 259200, np.inf]
dur_labels = ["0 to 1 min","1 to 10 min","10 to 60 min","1 to 6 hours","6 to 24 hours","1 to 3 days","over 3 days"]
job_df["duration_bucket"] = pd.cut(job_df["duration"], bins=dur_bins, labels=dur_labels, right=False)

gpu_bins = [0,1,2,5,9,17,33,65,129, np.inf]
gpu_labels = ["0","1","2 to 4","5 to 8","9 to 16","17 to 32","33 to 64","65 to 128","129 plus"]
job_df["gpu_demand_bucket"] = pd.cut(job_df["gpu_demand"], bins=gpu_bins, labels=gpu_labels, right=False, include_lowest=True)

wrk_bins = [1,2,9,33,129, np.inf]
wrk_labels = ["1","2 to 8","9 to 32","33 to 128","129 plus"]
job_df["worker_bucket"] = pd.cut(job_df["worker_num"], bins=wrk_bins, labels=wrk_labels, right=False, include_lowest=True)

ratio_bins = [0,1,2,4,8,16,32,64,128, np.inf]
ratio_labels = ["0 to 1","1 to 2","2 to 4","4 to 8","8 to 16","16 to 32","32 to 64","64 to 128","over 128"]
job_df["cpu_per_gpu_bucket"] = pd.cut(job_df["cpu_per_gpu"], bins=ratio_bins, labels=ratio_labels, right=False)

def bucket_table(col):
    tbl = (
        job_df.groupby(col)
        .agg(jobs=("gpu_seconds","count"),
             share_jobs=("gpu_seconds", lambda s: len(s)),
             gpu_seconds=("gpu_seconds","sum"))
        .sort_values("jobs", ascending=False)
    )
    total_jobs = tbl["jobs"].sum()
    total_gpu = tbl["gpu_seconds"].sum()
    tbl["jobs_share"] = tbl["jobs"] / max(total_jobs, 1)
    tbl["gpu_seconds_share"] = tbl["gpu_seconds"] / max(total_gpu, 1)
    return tbl

bucket_table("duration_bucket").to_csv(TBLDIR / "bucket_duration.csv")
bucket_table("gpu_demand_bucket").to_csv(TBLDIR / "bucket_gpu_demand.csv")
bucket_table("worker_bucket").to_csv(TBLDIR / "bucket_worker_num.csv")
bucket_table("cpu_per_gpu_bucket").to_csv(TBLDIR / "bucket_cpu_per_gpu.csv")

Computing bucketed summaries...


C:\Users\sulta\AppData\Local\Temp\ipykernel_33392\796056133.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  job_df.groupby(col)
C:\Users\sulta\AppData\Local\Temp\ipykernel_33392\796056133.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  job_df.groupby(col)
C:\Users\sulta\AppData\Local\Temp\ipykernel_33392\796056133.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  job_df.groupby(col)
C:\Users\sulta\AppData\Lo

In [17]:
# 7. Heatmaps for bivariate structure
print("Creating heatmaps...")
plt.figure()
plt.hist2d(job_df["gpu_request"], job_df["duration_hours"], bins=[50,50])
plt.xlabel("GPU request per worker")
plt.ylabel("Duration hours")
plt.title("GPU request versus duration heatmap")
plt.tight_layout()
plt.savefig(FIGDIR / "heatmap_gpu_request_vs_duration.png")
plt.close()

plt.figure()
plt.hist2d(job_df["worker_num"], job_df["duration_hours"], bins=[50,50])
plt.xlabel("Worker count")
plt.ylabel("Duration hours")
plt.title("Worker count versus duration heatmap")
plt.tight_layout()
plt.savefig(FIGDIR / "heatmap_workers_vs_duration.png")
plt.close()

Creating heatmaps...


In [18]:
# 8. Inter arrival times and burstiness
print("Computing inter arrival and burstiness...")
times = np.sort(job_df["submit_time"].values)
inter_arrival = np.diff(times)
inter_q = np.quantile(inter_arrival, [0.5,0.9,0.95,0.99]) if inter_arrival.size > 0 else np.array([np.nan]*4)
pd.DataFrame({"quantile":[0.5,0.9,0.95,0.99], "seconds":inter_q}).to_csv(TBLDIR / "inter_arrival_quantiles.csv", index=False)

counts_per_hour = np.bincount(start_hour, minlength=max_hour+1).astype(float)
mean_c = counts_per_hour.mean() if counts_per_hour.size > 0 else np.nan
var_c = counts_per_hour.var() if counts_per_hour.size > 0 else np.nan
fano = var_c / mean_c if mean_c and mean_c > 0 else np.nan
cv = np.sqrt(var_c) / mean_c if mean_c and mean_c > 0 else np.nan
with open(OUTDIR / "summary_extra.txt", "a", encoding="utf-8") as f:
    f.write(f"Inter arrival median seconds {float(inter_q[0]) if inter_arrival.size>0 else float('nan'):.2f}\n")
    f.write(f"Hourly arrival mean {mean_c:.2f} variance {var_c:.2f} fano {fano:.3f} cv {cv:.3f}\n")

plt.figure()
pd.Series(inter_arrival).clip(upper=np.quantile(inter_arrival, 0.99) if inter_arrival.size>0 else 1).plot(kind="hist", bins=60, title="Inter arrival seconds clipped at p99")
plt.xlabel("Seconds")
plt.tight_layout()
plt.savefig(FIGDIR / "inter_arrival_hist.png")
plt.close()

Computing inter arrival and burstiness...


In [19]:
# 9. Spot and HP risk slices if job_type exists
if "job_type" in job_df.columns:
    print("Computing Spot and HP slices...")
    tp = job_df["job_type"].fillna("Unknown")
    cuts = pd.DataFrame({
        "type": ["HP","Spot"],
        "jobs_share": [float((tp=="HP").mean()), float((tp=="Spot").mean())],
        "gpu_seconds_share": [
            float(job_df.loc[tp=="HP","gpu_seconds"].sum() / max(job_df["gpu_seconds"].sum(),1.0)),
            float(job_df.loc[tp=="Spot","gpu_seconds"].sum() / max(job_df["gpu_seconds"].sum(),1.0)),
        ],
    })
    cuts.to_csv(TBLDIR / "job_type_shares.csv", index=False)

    long_thr = 6*3600
    long_tbl = job_df.assign(is_long = job_df["duration"] >= long_thr).groupby(["is_long", "job_type"]).agg(
        jobs=("gpu_seconds","count"),
        gpu_seconds=("gpu_seconds","sum")
    ).reset_index()
    long_tbl.to_csv(TBLDIR / "long_job_slices_by_type.csv", index=False)

Computing Spot and HP slices...


In [20]:
# 10. Model mix per organization if columns exist
if job_model_col is not None and "organization" in job_df.columns:
    print("Computing model mix per organization...")
    org_model = job_df.pivot_table(index="organization", columns=job_model_col, values="gpu_seconds", aggfunc="sum").fillna(0.0)
    org_model_share = org_model.div(org_model.sum(axis=1).replace(0.0, np.nan), axis=0)
    org_model_share.to_csv(TBLDIR / "org_model_mix_share.csv")

# 11. Outlier finds
print("Finding outliers...")
short_heavy = job_df[(job_df["duration"] <= 300) & (job_df["gpu_demand"] >= 32)].copy()
keep_cols = [c for c in ["job_name","organization","job_type","duration","gpu_request","worker_num","gpu_demand","cpu_request","submit_time"] if c in job_df.columns]
short_heavy[keep_cols].sort_values("gpu_demand", ascending=False).head(500).to_csv(TBLDIR / "outliers_short_heavy.csv", index=False)

high_ratio = job_df[job_df["cpu_per_gpu"] >= 64].copy()
high_ratio[keep_cols + ["cpu_per_gpu"]].sort_values("cpu_per_gpu", ascending=False).head(500).to_csv(TBLDIR / "outliers_high_cpu_per_gpu.csv", index=False)

print("Extra analysis complete. Tables in", TBLDIR, "Figures in", FIGDIR, "Extra summary in", OUTDIR / "summary_extra.txt")

Computing model mix per organization...
Finding outliers...
Extra analysis complete. Tables in outputs_extra\tables Figures in outputs_extra\figures Extra summary in outputs_extra\summary_extra.txt
